# Chernobyl reactor n°2 :

## Set up

In [ ]:
# Basic libraries
import pandas as pd

# Streamlit app
import streamlit as st

# Visualization library
import plotly.express as px

# Syntax
import re

# Path managment
import os
import sys
sys.path.append('.')
sys.path.append('../src')

# Custom libraries
import config as c
import function as f

# Code aesthetic
#import tqdm
import warnings

warnings.filterwarnings('ignore')

In [50]:
d = {'Rk' : 'float64',
        'Player' : 'object',
        'Age' : 'float64',
        'Team' : 'object',
        'Pos' : 'object',
        'G' : 'int32',
        'GS' : 'int32',
        'MP' : 'float64',
        'FG' : 'float64',
        'FGA' : 'float64',
        'FG%' : 'float64',
        '3P' : 'float64',
        '3PA' : 'float64',
        '3P%' : 'float64',
        '2P' : 'float64',
        '2PA' : 'float64',
        '2P%' : 'float64',
        'eFG%' : 'float64',
        'FT' : 'float64',
        'FTA' : 'float64',
        'FT%' : 'float64',
        'ORB' : 'float64',
        'DRB' : 'float64',
        'TRB' : 'float64',
        'AST' : 'float64',
        'STL' : 'float64',
        'BLK' : 'float64',
        'TOV' : 'float64',
        'PF' : 'float64',
        'PTS' : 'float64',
        'Awards' : 'object',
        'Year' : 'int64'
    }

df = pd.read_csv(os.path.join("../", c.PROCESSED_DATA_PATH, c.PER_GAME_DF_FILENAME), dtype=d)

## App test

### Problem with [0-9]TM :

In [51]:
player = "Aaron Holiday"

player_df = df[df["Player"] == player]

# 2°/ Scrolling list for season
season = 2022

season_df = player_df[player_df["Year"] == season].iloc[0]

In [53]:
# https://www.geeksforgeeks.org/python/python-regex-cheat-sheet/

regex = r'[0-9]TM'
player_df['Team'].str.contains(regex)

2925    False
3417    False
4195    False
4959     True
4960    False
4961    False
5892    False
6382    False
7237    False
7789    False
Name: Team, dtype: bool

In [47]:
multi_team = (
    player_df.groupby(['Player', 'Year'])['Team']
      .transform('nunique') > 1
)
multi_team

2925    False
3417    False
4195    False
4959     True
4960     True
4961     True
5892    False
6382    False
7237    False
7789    False
Name: Team, dtype: bool

In [49]:
df_tm_only = player_df[
    (multi_team & (player_df['Team'].str.contains(regex))) |
    (~multi_team)
].copy()
df_tm_only

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
2925,319.0,Aaron Holiday,22.0,IND,PG,50,0,12.9,2.1,5.2,...,1.2,1.3,1.7,0.4,0.3,0.8,1.4,5.9,No Awards,2019
3417,187.0,Aaron Holiday,23.0,IND,PG,66,33,24.5,3.5,8.5,...,2.0,2.4,3.4,0.8,0.2,1.3,1.8,9.5,No Awards,2020
4195,278.0,Aaron Holiday,24.0,IND,PG,66,8,17.8,2.6,6.6,...,1.1,1.3,1.9,0.7,0.2,1.0,1.4,7.2,No Awards,2021
4959,323.0,Aaron Holiday,25.0,2TM,PG,63,15,16.2,2.4,5.4,...,1.6,1.9,2.4,0.7,0.1,1.1,1.5,6.3,No Awards,2022
5892,417.0,Aaron Holiday,26.0,ATL,PG,63,6,13.4,1.5,3.5,...,0.8,1.2,1.4,0.6,0.2,0.6,1.3,3.9,No Awards,2023
6382,278.0,Aaron Holiday,27.0,HOU,PG,78,1,16.3,2.4,5.3,...,1.3,1.6,1.8,0.5,0.1,0.7,1.6,6.6,No Awards,2024
7237,363.0,Aaron Holiday,28.0,HOU,PG,62,3,12.8,1.9,4.3,...,1.0,1.3,1.3,0.3,0.2,0.6,1.0,5.5,No Awards,2025
7789,277.0,Aaron Holiday,29.0,HOU,PG,27,0,14.2,2.1,5.1,...,0.7,0.9,1.0,0.4,0.1,0.8,1.3,6.5,No Awards,2026


In [ ]:
df_no_tm = player_df[
    ~(multi_team & (player_df['Team'].str.contains(regex)))
].copy()
df_no_tm

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
2925,319.0,Aaron Holiday,22.0,IND,PG,50,0,12.9,2.1,5.2,...,1.2,1.3,1.7,0.4,0.3,0.8,1.4,5.9,No Awards,2019
3417,187.0,Aaron Holiday,23.0,IND,PG,66,33,24.5,3.5,8.5,...,2.0,2.4,3.4,0.8,0.2,1.3,1.8,9.5,No Awards,2020
4195,278.0,Aaron Holiday,24.0,IND,PG,66,8,17.8,2.6,6.6,...,1.1,1.3,1.9,0.7,0.2,1.0,1.4,7.2,No Awards,2021
4960,323.0,Aaron Holiday,25.0,WAS,PG,41,14,16.2,2.4,5.2,...,1.4,1.6,1.9,0.6,0.2,1.0,1.5,6.1,No Awards,2022
4961,323.0,Aaron Holiday,25.0,PHO,PG,22,1,16.3,2.3,5.6,...,1.8,2.5,3.4,0.8,0.0,1.3,1.5,6.8,No Awards,2022
5892,417.0,Aaron Holiday,26.0,ATL,PG,63,6,13.4,1.5,3.5,...,0.8,1.2,1.4,0.6,0.2,0.6,1.3,3.9,No Awards,2023
6382,278.0,Aaron Holiday,27.0,HOU,PG,78,1,16.3,2.4,5.3,...,1.3,1.6,1.8,0.5,0.1,0.7,1.6,6.6,No Awards,2024
7237,363.0,Aaron Holiday,28.0,HOU,PG,62,3,12.8,1.9,4.3,...,1.0,1.3,1.3,0.3,0.2,0.6,1.0,5.5,No Awards,2025
7789,277.0,Aaron Holiday,29.0,HOU,PG,27,0,14.2,2.1,5.1,...,0.7,0.9,1.0,0.4,0.1,0.8,1.3,6.5,No Awards,2026
